In [1]:
!pip install pandas numpy sklearn matplotlib tensorly openpyxl keras scipy 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, FastICA, NMF
import tensorly as tl
from tensorly.decomposition import tucker,non_negative_tucker,parafac,non_negative_parafac
from tensorly import tucker_to_tensor
from tensorly.cp_tensor import cp_to_tensor
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import scipy.integrate as integrate
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.naive_bayes import GaussianNB

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
## Import Data here
X=pd.read_excel('Machine_Learning_tutorials\EMG_daven1.xlsx')
Y=pd.read_excel('Machine_Learning_tutorials\trg.xlsx')
print(X)
print(Y)


FileNotFoundError: [Errno 2] No such file or directory: 'Machine_Learning_tutorials\\EMG_daven1.xlsx'

In [ ]:
##Specify the model rank you want to investigate using this variable throughout
num_components=3


In [ ]:
##The task variables available here to predict are presented below:
display(Y)

##Select which variable you would like to predict by setting Type=1 for P1-P8 reaching tasks in 
##both forward and backwards directions, set Type=2 for the speed variable and Type=3 for the reaching diretion variable.

Type=2

if Type==1:
    task=Y.iloc[:,0]
elif Type==2:
    task=Y.iloc[:,1]
else:
    task=Y.iloc[:,2]

In [ ]:
##Non-negative matrix factorisation in both the spatial and temporal domains

##https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

##Here are the hyperparameters for NNMF that you can adjust, press shift+tab while inside the function to see the options
##in detail

# 'init': The method to initialise the extracted components
# 'solver': Numerical solver to use for the optimisation problem
# 'beta_loss': the loss function
# 'tol': the Tolerance of the stopping criterion
# 'max_iter': Maximum number of iterations during timing out
# 'alpha': a constant that adjust the influence of the regularisations terms in the loss function
# 'l1_ratio': a mixing parameter that allows one to adjust the regularisation term towards either L1 or L2 penaly terms


##What is the effect of adjusting each of these hyperparameters on the output???


model=NMF(n_components=num_components,solver='cd',
    beta_loss='frobenius',
    tol=0.0001,
    max_iter=200,
    random_state=None,
    alpha=0.0,
    l1_ratio=0.0,
    verbose=0,
    shuffle=False)


W_spatial = model.fit_transform(abs(X).T)
H_spatial = model.components_

pd.DataFrame(W_spatial).plot(kind='bar')
plt.show()
print('Timepoint specific activation coefficient :', H_spatial)

X_temp=X.values.reshape(640, 50, 9).transpose(1,0,2).reshape(50,640*9)

W_temporal=model.fit_transform(abs(X_temp))
H_temporal = model.components_


plt.plot(W_temporal)
plt.show()
print('Trial-specific muscle activation coefficient :', H_temporal)

In [ ]:
## Principal component analysis
## https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html


##Try what you did regarding the hyperparameters here on PCA...

model=PCA(n_components=num_components, copy=True, whiten=False, 
                          svd_solver='auto', tol=0.0, iterated_power='auto', random_state=None)

model.fit(X)
print('Explained variance ratio: ',model.explained_variance_ratio_)
print('Singular values: ', model.singular_values_)

X_pca=model.transform(X)

plt.plot(X_pca)

In [ ]:
#Independent component analysis
##https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html

model=FastICA(n_components=num_components, algorithm='parallel', whiten=True, 
        fun='logcosh', fun_args=None, max_iter=200, 
        tol=0.0001, w_init=None, random_state=None)
model.fit(X)
print('Components: ',model.components_)
print('Mixing values: ', model.mixing_)
print('Mean values: ', model.mean_)

X_ica=model.transform(X)

plt.plot(X_ica)

In [ ]:
##Tensor decompositions: Tucker decomposition
##http://tensorly.org/stable/user_guide/tensor_decomposition.html

X_tensor=tl.tensor(abs(X).values.reshape(640,50,9))

core, factors_tuck = tucker(X_tensor ,rank=[num_components,num_components,num_components])
tucker_recon=tucker_to_tensor([core, factors_tuck])

SSE=np.sum(np.linalg.norm((X_tensor-tucker_recon))**2)
SST=np.sum(np.linalg.norm(X_tensor)**2)
VAF_tuck=1-SSE/SST
print('Variance accounted for: ',VAF_tuck)

pd.DataFrame(factors_tuck[0]).plot()
pd.DataFrame(factors_tuck[1]).plot()
pd.DataFrame(factors_tuck[2]).plot(kind='bar')

In [ ]:
##Tensor decompositions: Non-negative Tucker
##http://tensorly.org/stable/user_guide/tensor_decomposition.html
X_tensor=tl.tensor(abs(X).values.reshape(640,50,9))
core, factors_nnt =non_negative_tucker(X_tensor ,rank=[num_components,num_components,num_components])

tucker_recon=tucker_to_tensor([core, factors_nnt])

SSE=np.sum(np.linalg.norm((X_tensor-tucker_recon))**2)
SST=np.sum(np.linalg.norm(X_tensor)**2)
VAF_nnt=1-SSE/SST
print('Variance accounted for: ',VAF_nnt)

pd.DataFrame(factors_nnt[0]).plot()
pd.DataFrame(factors_nnt[1]).plot()
pd.DataFrame(factors_nnt[2]).plot(kind='bar')

In [ ]:
##Tensor decompositions: PARAFAC Decomposition
##http://tensorly.org/stable/user_guide/tensor_decomposition.html
X_tensor=tl.tensor(abs(X).values.reshape(640,50,9))
factors_para = non_negative_parafac(X_tensor ,rank=num_components)
parafac_recon=tl.cp_to_tensor(factors_para)
SSE=np.sum(np.linalg.norm((X_tensor-parafac_recon))**2)
SST=np.sum(np.linalg.norm(X_tensor)**2)
VAF_para=1-SSE/SST
print('Variance accounted for: ',VAF_para)

pd.DataFrame(factors_para[1][0]).plot()
pd.DataFrame(factors_para[1][1]).plot()
pd.DataFrame(factors_para[1][2]).plot(kind='bar')

In [ ]:
## Decode the task using the underlying parameters trial-specific coefficients from the above tensor decompositions
## For the Task1-16 variable, a decoding accuracy >1/16 is significant and so on for the other variables.
## https://www.frontiersin.org/articles/10.3389/fncom.2013.00008



In [ ]:
#Tucker decomposition
X_train, X_test, y_train, y_test=train_test_split(factors_tuck[0],task,train_size=0.85,stratify=task)

model=LDA()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print('LDA Accuracy score: ',accuracy_score(y_test,y_pred))
LDA_acc_tuck=accuracy_score(y_test,y_pred)
plot_confusion_matrix(model,X_test,y_test)
plt.show()

#Naive-Bayes
#https://scikit-learn.org/stable/modules/naive_bayes.html
gnb = GaussianNB()
gnb.fit(X_train,y_train)
y_pred=gnb.predict(X_test)
print('Naive Bayes Accuracy score: ',accuracy_score(y_test,y_pred))
plot_confusion_matrix(gnb ,X_test,y_test)
plt.show()


##K-Nearest Neighbors
##https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
knn=KNN(n_neighbors=5,weights='distance')
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
print('K-Nearest Neighbors Accuracy score: ',accuracy_score(y_test,y_pred))
plot_confusion_matrix(knn ,X_test,y_test)
plt.show()

In [ ]:
##Non-negative Tucker decomposition
X_train, X_test, y_train, y_test=train_test_split(factors_nnt[0],task,train_size=0.85,stratify=task)

model=LDA()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print('LDA Accuracy score: ',accuracy_score(y_test,y_pred))
LDA_acc_nnt=accuracy_score(y_test,y_pred)
plot_confusion_matrix(model,X_test,y_test)
plt.show()

#Naive-Bayes
#https://scikit-learn.org/stable/modules/naive_bayes.html
gnb = GaussianNB()
gnb.fit(X_train,y_train)
y_pred=gnb.predict(X_test)
print('Naive Bayes Accuracy score: ',accuracy_score(y_test,y_pred))
plot_confusion_matrix(gnb ,X_test,y_test)
plt.show()


##K-Nearest Neighbors
##https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
knn=KNN(n_neighbors=5,weights='distance')
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
print('K-Nearest Neighbors Accuracy score: ',accuracy_score(y_test,y_pred))
plot_confusion_matrix(gnb ,X_test,y_test)
plt.show()

In [ ]:
##Parafac-decomposition
X_train, X_test, y_train, y_test=train_test_split(factors_para[1][0],task,train_size=0.85,stratify=task)

model=LDA()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print('LDA Accuracy score: ',accuracy_score(y_test,y_pred))
LDA_acc_para=accuracy_score(y_test,y_pred)
plot_confusion_matrix(model,X_test,y_test)
plt.show()

#Naive-Bayes
#https://scikit-learn.org/stable/modules/naive_bayes.html
gnb = GaussianNB()
gnb.fit(X_train,y_train)
y_pred=gnb.predict(X_test)
print('Naive Bayes Accuracy score: ',accuracy_score(y_test,y_pred))
plot_confusion_matrix(gnb ,X_test,y_test)
plt.show()


##K-Nearest Neighbors
##https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
knn=KNN(n_neighbors=5,weights='distance')
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)
print('K-Nearest Neighbors Accuracy score: ',accuracy_score(y_test,y_pred))
plot_confusion_matrix(knn ,X_test,y_test)
plt.show()

In [ ]:
##Can you find the model that gives the best output in terms of the VAF vs Task-decoding trade-off??
## https://www.frontiersin.org/articles/10.3389/fncom.2013.00008

print('Tucker decomposition: ', VAF_tuck*LDA_acc_tuck)
print('Non-negative Tucker decomposition: ', VAF_nnt*LDA_acc_nnt)
print('PARAFAC decomposition: ', VAF_para*LDA_acc_para)

In [ ]:
##Deep learning##

##Below you can find the code setup for a multi-layer perceptron (MLP_, the simplest form of neural network.
##Who's output layer is applicable to classification problems


##Pick the output from a tensor decomposition to use as input into the MLP, set Type=1 for Tucker, 2 for Non-negative tucker
##and 3 for PARAFAC. Note that the MLP is sensitive to the magnitude of the input variables and therefore it is necessary to
##standardise them


##Look across the web for a general rule for how many layers and nodes one should have in a neural network and see if it
##produces the best results here.


Type=1

if Type==1:
    X_mlp=factors_tuck[0]
elif Type==2:
    X_mlp=factors_nnt[0]
else:
    X_mlp=factors_para[1][0]

    
scaler= StandardScaler()


X_train, X_test, y_train, y_test=train_test_split(X_mlp,task.values,train_size=0.85)

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

clf = MLPClassifier(solver='adam', alpha=1e-5,
                     hidden_layer_sizes=(75,25), random_state=1)

clf.fit(X_train, y_train)

y_preds=clf.predict(X_test)
print('MLP Accuracy score: ',accuracy_score(y_test,y_preds))
plot_confusion_matrix(clf ,X_test,y_test)
plt.show()

In [ ]:
##Try to construct your own neural network that can classify the right reaching task...here is an example using the Keras framework
##https://keras.io/

##Pick the output from a tensor decomposition to use as input into the MLP, set Type=1 for Tucker, 2 for Non-negative tucker
##and 3 for PARAFAC. Note that the neural network is sensitive to the magnitude of the input variables and therefore it is necessary to
##standardise them

Type=3

if Type==1:
    X_NN=factors_tuck[0]
elif Type==2:
    X_NN=factors_nnt[0]
else:
    X_NN=factors_para[1][0]

    
##This neural network cannot take multiple classes as a target variable, therefore we must encode the target variable
##as a set of binary integers
# encode class values as integers


if task.name=='Task':
    encoder = LabelEncoder()
    encoder.fit(task)
    encoded_Y = encoder.transform(task)
    # convert integers to dummy variables (i.e. one hot encoded)
    dummy_y = np_utils.to_categorical(encoded_Y)
    X_train, X_test, y_train, y_test=train_test_split(X_NN,dummy_y,train_size=0.85)
else:
    X_train, X_test, y_train, y_test=train_test_split(X_NN,task.values,train_size=0.85)
    
##As with the MLP above, the neural network is sensitive to differences in magnitude between predictor variables that may not
##necessarily be informative...therefore we will scale the input data accordingly
scaler= StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)





##Here are some parameters you can adjust to optimise the neural network, how do these adjustments effect the output??:

#The number of neurons in the layer
#The number of layers
#The activation function
#The loss function
#The optimizer
#The metrics
#The training epochs
#The batch size
#The number of cross-validations




def NeuralNet_model():
# create model
    model = Sequential()
    model.add(Dense(50, input_dim=num_components, activation='relu'))
    #model.add(Dense(50, input_dim=num_components, activation='relu'))
    model.add(Dense(np.unique(task)[-1], activation='softmax'))
# Compile model
    model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train,y_train)
    return model

estimator = KerasClassifier(build_fn=NeuralNet_model, epochs=200, batch_size=1, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X_test, y_test, cv=kfold)
print("Cross validated score (Mean (Std)): %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))